In [1]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import pandas_datareader as web
from datetime import date
from datetime import datetime, timedelta, date


In [2]:
def predict_stock(ticker):
    year_ago = str(date.today()-timedelta(days=365))
    today = str(date.today())
    ticker = ticker
    source = 'yahoo'
    df = web.DataReader(ticker, data_source = source, start = year_ago, end = today)
    X = pd.DataFrame(df.drop(columns = ['Close']))
    y = pd.DataFrame(df['Close'])
    X = X.values
    y = y.values
    training_data_len = math.ceil(X.shape[0]*.99)
    scaler = MinMaxScaler(feature_range = (0,1))
    scaled_data = scaler.fit_transform(X)
    scaled_y = scaler.fit_transform(y)
    X = scaled_data[0:training_data_len, :]
    Y = scaled_y[0:training_data_len, :]
    X_train = []
    y_train = []
    for i in range(30, len(X)):
        X_train.append(X[i-30:i, :].ravel())
        y_train.append(Y[i, :])
    X_train, y_train = np.array(X_train), np.array(y_train)
    X_train = np.reshape(X_train, (X_train.shape[0],X_train.shape[1], 1))
    #build LSTM model
    model = Sequential()
    model.add(LSTM(50, return_sequences = True, input_shape = (X_train.shape[1],1)))
    model.add(LSTM(50, return_sequences = False ))
    model.add(Dense(25))
    model.add(Dense(1))
    model.compile(optimizer = 'adam', loss = 'mean_squared_error')
    model.fit(X_train, y_train, batch_size = 1, epochs = 1)
    test_data = scaled_data[training_data_len-30: , :]
    #create x_test, y_test
    X_test =[]
    y_test = y[training_data_len:,]
    for i in range(30, len(test_data)):
        X_test.append(test_data[i-30:i,:].ravel())
    X_test = np.array(X_test)
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1],1))
    predictions = model.predict(X_test)
    predictions = scaler.inverse_transform(predictions)
    rmse = np.sqrt(np.mean(predictions-y_test)**2)
    data = df.filter(['Close'])
    train= data[0:training_data_len]
    valid = data[training_data_len :]
    valid['Predictions'] = predictions
    close = df['Close'].values
    X_val = X[-30:,:].ravel()
    X_v = []
    X_v.append(X_val)
    X_v = np.array(X_v)
    X_v = np.reshape(X_v ,(X_v.shape[0],X_v.shape[1],1))
    tomorrow = model.predict(X_v)
    tomorrow = round(float(scaler.inverse_transform(tomorrow)),2)
    percent_from_close = round(float((tomorrow - close[-1:])/close[-1:])*100,2)
    percent_from_predict = round(float((tomorrow - valid['Predictions'].values[1])/valid['Predictions'].values[1])*100,2)
    return tomorrow, percent_from_close, percent_from_predict

In [3]:
my_stock_list = ['AAPL','GOOG','TSLA','AMZN','PXD','JD','UNH']
for item in my_stock_list:
    l = str(list(predict_stock(item)))
    print(item + ': ' +l)

1/1 [==============================] - 0s 31ms/step
AAPL: [140.97, 1.47, 0.69]


C:\Users\jyama\AppData\Local\Temp/ipykernel_6692/1392444293.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid['Predictions'] = predictions


1/1 [==============================] - 0s 33ms/step
GOOG: [2278.67, 4.45, 1.86]


C:\Users\jyama\AppData\Local\Temp/ipykernel_6692/1392444293.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid['Predictions'] = predictions


1/1 [==============================] - 0s 31ms/step
TSLA: [762.45, 11.83, 2.07]


...

C:\Users\jyama\AppData\Local\Temp/ipykernel_6692/1392444293.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid['Predictions'] = predictions


1/1 [==============================] - 0s 30ms/step
JD: [65.77, -0.27, 1.45]


C:\Users\jyama\AppData\Local\Temp/ipykernel_6692/1392444293.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid['Predictions'] = predictions


1/1 [==============================] - 0s 34ms/step
UNH: [501.61, -3.05, -0.05]


C:\Users\jyama\AppData\Local\Temp/ipykernel_6692/1392444293.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid['Predictions'] = predictions
